In [1]:
import pandas as pd
import os
from glob import glob

# 📌 선박 밀집도 데이터 폴더 경로 설정
ship_data_folder = "sden_2023_lv3"  # 폴더명 변경 가능

# 📌 모든 CSV 파일 리스트 가져오기
ship_files = sorted(glob(os.path.join(ship_data_folder, "sden_2023*.csv")))

# 📌 데이터프레임 리스트
ship_dataframes = []

# 📌 모든 선박 밀집도 데이터 로드 및 통합
for file_path in ship_files:
    # 파일명에서 날짜/시간 정보 추출
    filename = os.path.basename(file_path)  # 파일명만 추출
    datetime_str = filename.split("_")[1]  # 예: "202301010100"
    datetime_parsed = pd.to_datetime(datetime_str, format="%Y%m%d%H%M")  # 날짜 변환

    # CSV 파일 로드 (쉼표 포함된 위경도 컬럼 처리)
    df = pd.read_csv(file_path)

    # 📌 컬럼명 표준화
    df.rename(columns={
        "격자번호": "Grid_ID",
        "위경도": "LatLon",
        "교통량(척)": "Ship_Count",
        "밀집도(%)": "Density",
        "어선": "Fishing",
        "여객선": "Passenger",
        "화물선": "Cargo",
        "유조선": "Tanker",
        "예인선": "Tugboat",
        "수상레저기구": "Leisure",
        "기타선": "Other"
    }, inplace=True)

    # 📌 위경도 분리 (쉼표 포함된 문자열에서 위도, 경도 분리)
    df[["Latitude", "Longitude"]] = df["LatLon"].str.split(", ", expand=True).astype(float)

    # 📌 datetime 컬럼 추가
    df["datetime"] = datetime_parsed

    # 📌 필요한 컬럼만 선택
    required_columns = ["Grid_ID", "Latitude", "Longitude", "datetime", "Ship_Count", "Density", 
                        "Fishing", "Passenger", "Cargo", "Tanker", "Tugboat", "Leisure", "Other"]
    df = df[required_columns]

    # 📌 데이터프레임 리스트에 추가
    ship_dataframes.append(df)

# 📌 모든 데이터프레임을 하나로 병합
ship_df = pd.concat(ship_dataframes, ignore_index=True)

# 📌 Grid_ID 및 시간별 평균 선박 밀집도 계산
ship_avg = ship_df.groupby(["Grid_ID", "datetime"]).agg(
    avg_ship_density=("Density", "mean"),
    avg_ship_count=("Ship_Count", "mean"),
    avg_fishing=("Fishing", "mean"),
    avg_passenger=("Passenger", "mean"),
    avg_cargo=("Cargo", "mean"),
    avg_tanker=("Tanker", "mean"),
    avg_tugboat=("Tugboat", "mean"),
    avg_leisure=("Leisure", "mean"),
    avg_other=("Other", "mean")
).reset_index()

# 📌 결과 저장
output_path = "ship_density_aggregated_2023.csv"
ship_avg.to_csv(output_path, index=False)

print("📌 선박 밀집도 데이터 통합 완료! ✅")
print(f"결과 파일: {output_path}")


📌 선박 밀집도 데이터 통합 완료! ✅
결과 파일: ship_density_aggregated_2023.csv
